# Prompt engineering

OpenAI promp engineering for using API requests to extract features from the news collected.

Using openai=2.1.0

Using structured output with BaseModel


In [24]:
from __future__ import annotations
from typing import List, Optional, Literal
from pydantic import BaseModel, Field
import pandas as pd
from pandas import json_normalize

from openai import OpenAI

project_path = "C:/Users/santt/Desktop/DataMining_UBA/5-text_mining/nlp_dmuba/"

### Loading dataset

In [20]:
dataset_file_path = project_path+"1-Scraping/dataset_consolidado/df.parquet" 
df = pd.read_parquet(dataset_file_path)

In [13]:
df.head()

,diario,fecha,titulo,contenido,url,seccion
0,Ámbito Financiero,2025-01-01,La reflexión de Javier Milei: el pronóstico pa...,Arrancó el 2025 y el presidente Javier Milei h...,https://www.ambito.com/politica/la-reflexion-j...,politica
1,Ámbito Financiero,2025-01-01,Wall Street apuesta por Wall Street: los más o...,Wall Street cerró el año con leves tomas de ga...,https://www.ambito.com/economia/wall-street-ap...,economia
2,Ámbito Financiero,2025-01-01,Karina Milei a los libertarios por Año Nuevo: ...,"La secretaria general de la Presidencia, Karin...",https://www.ambito.com/politica/karina-milei-l...,politica
3,Ámbito Financiero,2025-01-01,"El mensaje de Javier Milei por Año Nuevo: ""Se ...",El presidente Javier Milei compartió un mensaj...,https://www.ambito.com/politica/el-mensaje-jav...,politica
4,Ámbito Financiero,2025-01-01,El ajuste alcanzó a las fiestas: las ventas de...,El 2024 fue un año marcado por un fuerte recor...,https://www.ambito.com/economia/el-ajuste-alca...,economia


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15244 entries, 0 to 15243
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   diario     15244 non-null  object        
 1   fecha      15244 non-null  datetime64[ns]
 2   titulo     15244 non-null  object        
 3   contenido  15207 non-null  object        
 4   url        15244 non-null  object        
 5   seccion    15244 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 714.7+ KB


In [15]:
# taking a sample of 10 news
df_sample = df.sample(10, random_state=42)

In [28]:
df_sample.head()

,diario,fecha,titulo,contenido,url,seccion
4093,Ámbito Financiero,2025-04-15,El gobierno de Javier Milei eliminó el RUCA y ...,La Secretaría de Agricultura resolvió dar de b...,https://www.ambito.com/economia/el-gobierno-ja...,economia
10766,La Nación,2025-03-03,Por qué la polarización favorece a Kicillof co...,El analista Rosendo Fraga examinó este domingo...,https://www.lanacion.com.ar/politica/por-que-l...,Política
10375,La Nación,2025-02-25,Banco Santander le compró a Grupo Galicia el 5...,"Banco Santander SA compró el 50% de Nera, la p...",https://www.lanacion.com.ar/economia/campo/ban...,Campo
3774,Ámbito Financiero,2025-04-07,Peligra en Diputados la sesión por $LIBRA: la ...,La oposición en Diputados aúna fuerzas para po...,https://www.ambito.com/politica/peligra-diputa...,politica
8779,La Nación,2025-01-29,La baja de retenciones podría ayudar a contene...,La reducción transitoria de las retenciones al...,https://www.lanacion.com.ar/economia/esquema-8...,Economía


### OpenAI Requests

Initiating OpenAI client

In [26]:
try:
    with open(project_path+'.secrets/openai_api_key.txt', 'r') as file:
        key = file.read()
except FileNotFoundError:
    print("Error: The file 'openai_api_key.txt' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

client = OpenAI(api_key = key)

Structure request

In [ ]:
model = "gpt-5-mini"

In [ ]:
class Evidencia(BaseModel):
    citas_evidencia: List[str] = Field(description="Hasta 3 frases textuales del artículo")
    notas: Optional[str] = Field(default=None, description="Resumen factual breve (sin especulación)")

class ImpactoDireccional(BaseModel):
    direccion: float = Field(description="[-1..1] Dirección esperada")
    magnitud: float = Field(description="[0..1] Magnitud esperada")

class SeñalesMercado(BaseModel):
    merval: ImpactoDireccional
    fx_ars: ImpactoDireccional
    tasa_bcra: ImpactoDireccional
    bonos_soberanos: ImpactoDireccional

class Optimismos(BaseModel):
    positividad_general: float
    apoyo_gobernanza: float
    optimismo_macro_corto: float
    optimismo_macro_largo: float
    optimismo_fin_corto: float
    optimismo_fin_largo: float

class Tematicas(BaseModel):
    menciona_inflacion: bool
    menciona_pbi: bool
    menciona_reservas: bool
    menciona_embi: bool
    menciona_deuda: bool
    menciona_fmi: bool
    menciona_salarios_paritarias: bool

class Entidades(BaseModel):
    tipo_actor_principal: Literal[
        "gobierno_nacional","bcra","provincia","municipio",
        "empresa_mercado_local","empresa_mercado_ext",
        "sindicato","poder_judicial","congreso","organismo_internacional",
        "desconocido"
    ]
    nombre_actor_principal: Optional[str] = "unknown"
    empresas_mencionadas: List[str] = []
    tickers_mencionados: List[str] = []
    sectores_mencionados: List[str] = []

class Evento(BaseModel):
    tipo_evento: Literal[
        "monetario","fiscal","regulatorio","corporativo","externo",
        "sindical_social","judicial","electoral","otro","desconocido"
    ]
    shock: Literal["positivo","negativo","mixto","neutro","desconocido"]
    caracter: Literal["retroactivo","vigente","prospectivo","desconocido"]
    horizonte_dias: Optional[int] = None

class CalidadFuente(BaseModel):
    categoria: Literal["oficial","periodistica","rumor","analisis","desconocido"]
    calidad_fuente_score: float

class NewFeatures(BaseModel):
    # Metadatos
    diario: Optional[str]
    fecha: Optional[str]
    seccion: Optional[str]
    url: Optional[str]

    # Núcleo
    entidades: Entidades
    evento: Evento
    mercado: SeñalesMercado
    optimismos: Optimismos
    tematicas: Tematicas

    # Calidad / confianza
    calidad_fuente: CalidadFuente
    confianza: float

    # Evidencia
    evidencia: Evidencia

SYSTEM_PROMPT = """Eres un analista económico-financiero especializado en Argentina.
Objetivo: extraer datos estructurados y auditables para modelar el MERVAL a partir de noticias.
Instrucciones:
- Usa SOLO la información presente en el texto de la noticia.
- Si la evidencia es insuficiente, devuelve "unknown" donde corresponda y establece confianza baja (<=0.3).
- Escalas: dirección [-1..1], magnitud [0..1], optimismos/apoyo [0..1], confianza [0..1].
- Horizonte en días (entero) si el texto sugiere timing; en caso contrario "unknown".
- Calidad de fuente: oficial/comunicado (0.9–1), periodística (0.6–0.8), análisis (0.5–0.7), rumor (0.2–0.4).
- Devuelve hasta 3 frases textuales como 'citas_evidencia' que justifiquen tus etiquetas (sin razonamientos internos).
- No inventes valores, no cites fuentes externas al artículo.
"""

USER_TEMPLATE = """Diario: {diario}
Fecha: {fecha}
Seccion: {seccion}
Titulo: {titulo}
Contenido: {contenido}
URL: {url}

Tareas:
1) Identifica actor principal y tipo.
2) Clasifica el tipo de evento, shock y carácter temporal.
3) Estima impacto direccional y magnitud esperada en: MERVAL, FX ARS, tasa BCRA, bonos soberanos.
4) Puntúa positividad general, apoyo a la gobernanza y optimismos macro/fin (corto/largo).
5) Marca temáticas macro (inflación, PBI, reservas, EMBI, deuda, FMI, salarios/paritarias).
6) Extrae empresas, sectores y tickers mencionados si existieran.
7) Estima horizonte_dias (si corresponde), calidad de fuente y confianza.
8) Devuelve citas_evidencia (máx. 3) y un breve resumen factual en 'notas'.
Devuelve salida en el esquema dado.
"""

results = []
for _, row in df_sample.iterrows():
    # Manejo de faltantes
    contenido = (row.get("contenido") or "")[:8000]
    prompt = USER_TEMPLATE.format(
        diario=row.get("diario", "unknown"),
        fecha=str(row.get("fecha", "unknown")),
        seccion=row.get("seccion", "unknown"),
        titulo=row.get("titulo", "unknown"),
        contenido=contenido,
        url=row.get("url", "unknown"),
    )

    resp = client.responses.parse(
        model=model,
        input=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
        text_format=NewFeatures,
    )

    parsed = resp.output_parsed
    # Completar metadatos desde el row si el modelo dejó "unknown"
    parsed.diario = parsed.diario or row.get("diario")
    parsed.fecha = parsed.fecha or str(row.get("fecha"))
    parsed.seccion = parsed.seccion or row.get("seccion")
    parsed.url = parsed.url or row.get("url")
    results.append(parsed.model_dump())

df_features = json_normalize(results, sep="__")

Took ~ 8 min for 10 news

In [31]:
df_features

,diario,fecha,seccion,url,mercado__merval__direccion,mercado__merval__magnitud,mercado__fx_ars__direccion,mercado__fx_ars__magnitud,mercado__tasa_bcra__direccion,mercado__tasa_bcra__magnitud,...,entidades__sectores_mencionados,tematicas__menciona_inflacion,tematicas__menciona_pbi,tematicas__menciona_reservas,tematicas__menciona_embi,tematicas__menciona_deuda,tematicas__menciona_fmi,tematicas__menciona_salarios_paritarias,evidencia__citas_evidencia,evidencia__notas
0,Ámbito Financiero,2025-04-15 00:00:00,economia,https://www.ambito.com/economia/el-gobierno-ja...,0.0,0.00,0.0,0.00,0.0,0.0,...,"[agro, cadena agroindustrial, cárnico, lácteo]",False,False,False,False,False,False,False,[La Secretaría de Agricultura resolvió dar de ...,La Secretaría de Agricultura (con ARCA) elimin...
1,La Nación,2025-03-03 00:00:00,Política,https://www.lanacion.com.ar/politica/por-que-l...,0.0,0.00,0.0,0.00,0.0,0.0,...,[],False,False,False,False,False,False,False,"[A partir de esto que está pasando, [Axel Kici...",El analista Rosendo Fraga afirmó que la polari...
2,La Nación,2025-02-25 00:00:00,Campo,https://www.lanacion.com.ar/economia/campo/ban...,0.1,0.15,0.0,0.00,0.0,0.0,...,"[agro, fintech, bancario, plataformas de pagos...",False,False,False,False,False,False,False,"[Banco Santander SA compró el 50% de Nera, la ...",Banco Santander SA adquirió el 50% de Nera (pl...
3,Ámbito Financiero,2025-04-07 00:00:00,politica,https://www.ambito.com/politica/peligra-diputa...,-0.2,0.15,0.2,0.15,0.0,0.0,...,[criptomonedas],False,False,False,False,False,False,False,[La oposición en Diputados aúna fuerzas para p...,La oposición en Diputados pidió sesionar este ...
4,La Nación,2025-01-29 00:00:00,Economía,https://www.lanacion.com.ar/economia/esquema-8...,0.2,0.20,-0.6,0.60,-0.3,0.3,...,"[agropecuario, productos agrícolas, economías ...",False,False,True,False,False,False,False,[La reducción transitoria de las retenciones a...,El Gobierno bajó transitoriamente las retencio...
5,La Nación,2025-01-14 00:00:00,Política,https://www.lanacion.com.ar/politica/milei-gan...,0.0,0.00,0.0,0.00,0.0,0.0,...,[],False,False,False,False,False,False,False,[El presidente Javier Milei ganó este martes e...,Javier Milei recibió el Genesis Prize (denomin...
6,Clarín,2025-04-05 00:00:00,economia,https://www.clarin.com/economia/historia-llevo...,0.0,0.00,0.0,0.00,0.0,0.0,...,"[acero, manufactura, comercio internacional, f...",False,False,False,False,True,False,False,[“Quiero un decreto para el viernes que retire...,Artículo periodístico que relata cómo Bill Cli...
7,Ámbito Financiero,2025-03-25 00:00:00,opinion,https://www.ambito.com/opiniones/dnu-necesidad...,-0.6,0.50,-0.7,0.60,0.5,0.5,...,[textil],False,False,True,False,True,True,False,[El DNU 179/2025 ... impone un nuevo endeudami...,Columna de opinión que critica el Decreto (DNU...
8,Ámbito Financiero,2025-03-02 00:00:00,politica,https://www.ambito.com/politica/facundo-manes-...,-0.3,0.25,-0.3,0.20,0.2,0.2,...,[],False,False,False,False,False,False,False,"[""Frenemos antes de que sea tarde la pulsión a...",El artículo narra un conflicto político: el di...
9,Clarín,2025-01-29 00:00:00,economia,https://www.clarin.com/economia/sectores-produ...,-0.5,0.40,0.0,0.00,0.0,0.0,...,[],False,True,False,False,False,False,False,[La inversión pública ejecutada durante 2024 a...,El Informe de la Oficina de Presupuesto del Co...


In [32]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 39 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   diario                                   10 non-null     object 
 1   fecha                                    10 non-null     object 
 2   seccion                                  10 non-null     object 
 3   url                                      10 non-null     object 
 4   mercado__merval__direccion               10 non-null     float64
 5   mercado__merval__magnitud                10 non-null     float64
 6   mercado__fx_ars__direccion               10 non-null     float64
 7   mercado__fx_ars__magnitud                10 non-null     float64
 8   mercado__tasa_bcra__direccion            10 non-null     float64
 9   mercado__tasa_bcra__magnitud             10 non-null     float64
 10  mercado__bonos_soberanos__direccion      10 non-null 

In [33]:
df_features.to_csv(project_path+"5-LLMs/openai/df_sample.csv", index=False)

To decide and improve:

- Avoiding quotes and other kind of feature requirend using more output tokens
- Avoiding reducing too complex features
- Aggregating news by day / by newspaper
